In [14]:
import argparse
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding as as_padding
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import load_pem_public_key, load_pem_private_key
import os
import random

### 1. Генерация ключей гибридной системы

In [48]:
way = os.path.abspath('')
file_settings= {
    'initial_file': way + '\\' + 'initial_file.txt',
    'encrypted_file': way + '\\' + 'encrypted_file.txt',
    'decrypted_file': way + '\\' + 'decrypted_file.txt',
    'symmetric_key': way + '\\' + 'symmetric_key.txt',
    'public_key': way + '\\' + 'public_key.pem',
    'private_key': way + '\\' + 'private_key.pem',
}

keys = [i for i in range(5, 17, 1)]


генерация ключа для симметричного алгоритма

In [49]:
def gen_keys(file_settings: dict, keys: list):
    len_key = random.randint(0, len(keys) - 1)
    key = os.urandom(keys[len_key])
    print('Ключ симметричного шифрования:')
    print(type(key))
    print(key)
    
    assym_keys = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    private_key = assym_keys
    public_key = assym_keys.public_key()
    
    print('\nЗакрытый ключ ассимметричного шифрования:')
    print(type(private_key))
    print(private_key)
    print('\nОтрытый ключ ассимметричного шифрования:')
    print(type(public_key))
    print(public_key)
    
    c_key = public_key.encrypt(key, as_padding.OAEP(mgf=as_padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    print('\nЗашифрованный ключ симметричного шифрования')
    print(c_key)
    
    with open(file_settings['public_key'], 'wb') as public_out:
            public_out.write(public_key.public_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo))

    with open(file_settings['private_key'], 'wb') as private_out:
            private_out.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()))
            
    with open(file_settings['symmetric_key'], 'wb') as key_file:
        key_file.write(c_key)

генерация пары ключей для acсимметричного алгоритма

In [50]:
gen_keys(file_settings, keys)

Ключ симметричного шифрования:
<class 'bytes'>
b'\xe7\x12\xa6\xc6\xab1'

Закрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey'>

Отрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPublicKey'>

Зашифрованный ключ симметричного шифрования
b'\x1c\x0f\x84\x8b\x92\xf5W\x8d\x91\xdb\'\x1a\xbe\xa5\xacq%\xa4\x00\x9fQ\xc0\xed\xf2z* H\xfb\xde\x00\xdd\xbdwA*%\x08\xb7l\xf63\xc3\xc0\xe4H\xa1\xaa\x8a\x99\x02\x8a"vU\xba\xbe/\xa7~0z\x8f\xc1[\x8a\xe8\xad\x029\xc2\xfa\xb1\xfce\xac\xa8A\xde$\x9c\xa8l\xe2r\xac}\x97\x1da\xae\xe8\xcc\xdc7\xf4)\xe2\xd4\x95\x9fn\xac\xf0y\xd9\xda\xe8\x12\xcc^S\xe8\xec\x15\xde\xf4\x98K2;\x8d\xd9I\xa2\xd08V\xc0\xcex=\xbc\x9e#\x0eS\x1d#0\x1e]\xca\xc2\x04\xe2j\xeb\xe8k;\xd1\xe9L\x0e\x7f\xd8\xde\xf2\x87\r\xdbGg\x0f\x95{\xb5\xc1^\xce-\xbc\xce\x9b\x12ra\x19\x15\xda\xeb\xa8\xc1$\x057\xf65\xa2,apV\xe0yD--c\x8c\xb9\x16\x91\xbf?$6\xc1\xbf\xf8#\xf7^\xf5d\x17\xdd\x93\xf5\xd5\xb0\xed\xc7R\x0cL\xd4

### 2. Шифрование данных гибридной системой

In [58]:
def sym_key_decryption(file_settings: dict):
    with open(file_settings['private_key'], 'rb') as file:
        private_bytes = file.read()
    d_private_key = load_pem_private_key(private_bytes,password=None,)
    print('\nЗасшифрованный ассимметричный ключ:', d_private_key)

    with open(file_settings['symmetric_key'], 'rb') as file:
        sym_key = file.read()
        
    print('Ключ симметричного шифрования:', sym_key)
  
    dec_sym_key = d_private_key.decrypt(sym_key,as_padding.OAEP(mgf=as_padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    
    print('\nРасшифрованный люч симметричного шифрования:', dec_sym_key)
    
    return dec_sym_key
    
    

In [59]:
def text_encryption(file_settings: dict):
    with open(file_settings['initial_file'], 'r', encoding = 'UTF-8') as file:
        text = file.read()
    print('Исходный текст:\n', text)
    
    dec_sym_key = sym_key_decryption(file_settings)
    
    padder = sym_padding.ANSIX923(32).padder()
    padded_text = padder.update(bytes(text, 'UTF-8'))+padder.finalize()

    iv = os.urandom(8) #случайное значение для инициализации блочного режима, должно быть размером с блок и каждый раз новым
    cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    enc_text = encryptor.update(padded_text) + encryptor.finalize()
    
    print('зашифрованный текст')
    print(enc_text)
    
    with open(file_settings['encrypted_file'], 'wb') as file:
        file.write(enc_text)

In [60]:
text_encryption(file_settings)

Исходный текст:
 КОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ Мы, многонациональный народ Российской Федерации, соединенные общей судьбой на своей земле, утверждая права и свободы человека, гражданский мир и согласие, сохраняя исторически сложившееся государственное единство, исходя из общепризнанных принципов равноправия и самоопределения народов, чтя память предков, передавших нам любовь и уважение к Отечеству, веру в добро и справедливость, возрождая суверенную государственность России и утверждая незыблемость ее демократической основы, стремясь обеспечить благополучие и процветание России, исходя из ответственности за свою Родину перед нынешним и будущими поколениями, сознавая себя частью мирового сообщества, принимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.

Засшифрованный ассимметричный ключ: <cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey object at 0x000001F3BFD31A60>
Ключ симметричного шифрования: b'\x1c\x0f\x84\x8b\x92\xf5W\x8d\x91\xdb\'\x1a\xbe\xa5\xacq%\xa4\x00\x9fQ\xc0\xed\xf2z* H\x

C:\Users\Marishka\AppData\Local\Temp\ipykernel_14892\507232674.py:12: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))


### 3. Дешифрование данных гибридной системой

In [63]:
def text_decryption(file_settings: dict):
    with open(file_settings['encrypted_file'], 'rb') as file:
        enc_text = file.read()
    print('Зашифрованный текст:\n', enc_text)
    
    iv = os.urandom(8)
    dec_sym_key = sym_key_decryption(file_settings)
    cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))
    
    decryptor = cipher.decryptor()
    dc_text = decryptor.update(enc_text) + decryptor.finalize()

    unpadder = sym_padding.ANSIX923(32).unpadder()
    unpadded_dc_text = unpadder.update(dc_text) + unpadder.finalize()

    print(dc_text.decode('UTF-8'))
    print(unpadded_dc_text.decode('UTF-8'))

In [64]:
text_decryption(file_settings)

Зашифрованный текст:
 b'\x9e\xd4e\xd4\x19\x9a\x04Z\xda\x1b\x16\x88\x05\xa7\xdd\x02\xf8\xd4\xc6\x19\xf7\xee\xbfI\x9e\x90\xf2wl\x95c\\\xae\x0bCP#\x9ce=\x04\xe0a\xcf-\xbc$\x11\xaa\x14\'\xeaM}%c\x022E\xed+\xb94\x10\x8a\xa9\xa0XE\x1f\xc69\xb7\x95*\x01\xea\xfb1\xdaU\xff\x7f\xa4\x06\xa1-ZD\x9f\xf9\t\xcf\xf9.\xf1\x98i\xbc\xa0\x8f\xd6K\\\xff\x7f`n\xff\xb8\xf0\xbd^"\x1b\xa4\x90a\x99\xaa\xe8r\x02\x1d\x1b\xc0\xacQ&9\xbb\x0e\xc7\xb8\x1b\xd7\x00\xf2\x9e1\xbbAIu\x876A\xf2j\xfd%r\x1a\xd5\xdc\xf7\xc45XL\xa4\xc2\xa63\xff0\x89\xb3\xc5Y\xf6\x9b\x8eOn9}\x95\x9a\xa2?\x07$&\xca\xb1\xff\xd9X\x8c\xcd\x1ba\x9b\x94\xbc\xd2\xae\xfb\xa3_\x1f/\xfa\xf6\x00\xe3\x13;\xb48\xe9\xf1\xe6\xf4\xdd\xc5\xea\x14\x8f\xfc(\x1a<\xf9\xaeF.c\xa6\x1c\xe4\x0c\x94\xb3\x90u\xf7\xd3<\xa2)\x99\xfc\x9e\xc6"\x97\xd5\xb6c\xe5\x95*Q_\x1d\x808\xc5J:(dY\x11\x91?w\xe6\x19\x7f\x11\xb1\xd2\xebR\xae\xa3\x17C\xce\x06\xcd\xf1PI<\x9fi\x1f\xcc\xc5z\xd1w\xa6\x94\x17\xd5R[C<\xd2\xf3Wja\xde\r\x06\xef\xa8\xf4#\xe1\x98,\x9fQ&@?\xc9%\xf4\xe5P\x89\xcf\x0e\xf

C:\Users\Marishka\AppData\Local\Temp\ipykernel_14892\440339411.py:8: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf5 in position 2: invalid start byte